# Stock data using MongoDB

In [1]:
from pymongo import MongoClient
import pandas as pd
import pymongo
import requests
from io import BytesIO
from textblob import TextBlob
import datetime
import secrets
import yfinance as yf

We are creating a database in MongoDB, that contains two collections (history of the Stock Prices and the News about our companies)

## Import Data News from seekingalpha using Selenium

In [2]:
msft_news = pd.read_csv('C:/Users/utilisateur/Downloads/IA simplon/Projets/Arbitrage_en_Bourse/CSV/msft_news_clean.csv')
goog_news = pd.read_csv('C:/Users/utilisateur/Downloads/IA simplon/Projets/Arbitrage_en_Bourse/CSV/goog_news_clean.csv')
amzn_news = pd.read_csv('C:/Users/utilisateur/Downloads/IA simplon/Projets/Arbitrage_en_Bourse/CSV/amzn_news_clean.csv')

In [3]:
msft_news = msft_news.drop("Unnamed: 0", axis = 1)
goog_news = goog_news.drop("Unnamed: 0", axis = 1)
amzn_news = amzn_news.drop("Unnamed: 0", axis = 1)

In [21]:
amzn_news.head()

,Date,Title
0,2020-10-01,"Amazon Music partners with Universal Music, Wa..."
1,2020-09-22,"Amazon to expand tech hubs, corporate offices,..."
2,2020-09-22,"Amazon hires former Yahoo, Slack privacy head ..."
3,2020-09-16,"Amazon's latest job spree: Another 100,000 wor..."
4,2020-08-26,Amazon targeted by German antitrust probe


## Import Data to Mongodb - Collection History stock prices from Yahoo finance API 

In [5]:
client = MongoClient("mongodb://localhost:27017/")
# create database
db = client["Strategies_trading"]
# collection
prices_history = db["prices_history"]
print(prices_history)
    
for ticker in ["MSFT", "AMZN", "GOOG"]:    
    yf_stock = yf.Ticker(ticker)
    history = yf_stock.history(period="5y")[["Open","Close"]]
    history["DailyChangePct"] = (history["Close"] / history["Open"])-1
    history['Date'] = history.index
    history["Ticker"] = ticker
    prices_history.insert_many(history.to_dict(orient = 'records'))

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Strategies_trading'), 'prices_history')


## Import Data to Mongodb - Collection News from seekingalpha using Selenium

In [6]:
client = MongoClient("mongodb://localhost:27017/")
# create database
db= client["Strategies_trading"]
# create collection
news_history = db["news_history"]
print(news_history)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Strategies_trading'), 'news_history')


In [ ]:
for price to ["msft_news", "amzn_news", "goog_news"]

### Amazon 

In [18]:
# csv to dictionnary
amzn_dict = amzn_news.to_dict("records")
amzn_dict[0:6]


[{'Date': '2020-10-01',
  'Title': 'Amazon Music partners with Universal Music, Warner Music on Ultra HD remasters'},
 {'Date': '2020-09-22',
  'Title': 'Amazon to expand tech hubs, corporate offices, adding 3,500 jobs'},
 {'Date': '2020-09-22',
  'Title': 'Amazon hires former Yahoo, Slack privacy head for Alexa effort - BI'},
 {'Date': '2020-09-16',
  'Title': "Amazon's latest job spree: Another 100,000 workers"},
 {'Date': '2020-08-26', 'Title': 'Amazon targeted by German antitrust probe'},
 {'Date': '2020-08-27',
  'Title': 'Amazon orders 1,800 Mercedes electric vans for European deliveries'}]

In [19]:
news_history.insert_one({"index":"Amazon News","data":amzn_dict})

### Microsoft

In [20]:
msft_dict = msft_news.to_dict("records")
news_history.insert_one({"index":"Microsoft News","data":msft_dict})

### Google

In [13]:
goog_dict = goog_news.to_dict("records")
news_history.insert_one({"index":"Google News","data":goog_dict})